In [1]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [2]:
import altair as alt

In [9]:
from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [10]:
import intake

In [11]:
pd.set_option("display.precision", 2)

In [24]:
# read directly from google sheet instead of this
#df = to_snakecase(pd.read_excel('OFFICIAGrantsOverviewandWalkthrough.xlsx'))

In [55]:
sheet_id = "12pw6_2OSHKGksnLQlvwvo6P8VljzQ9pse7E9zGDSpg4"
sheet_name = "current_grants"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = to_snakecase(pd.read_csv(url))

In [56]:
df

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,unnamed:_27,unnamed:_28,unnamed:_29,unnamed:_30,unnamed:_31,unnamed:_32,unnamed:_33,unnamed:_34,unnamed:_35,unnamed:_36
0,1,SB1,5304 Metropolitan & Statewide Planning and Non...,Missing,Missing,Missing,1 Year,3000000,MPOs/RTPAs\n• Transit Agencies\n• Universities...,Formula,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Caltrans - Unknown Division,Capital Investment Grants - 5309,Missing,Missing,Missing,1 Year,2300000000,Large Urban MPOs and Transit Agencies,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Clean CA,Clean California Local Grant Program,"December 1, 2021","February 1, 2022",2,Single,296000000,"Transit Agency, Tribal Government, Municipalit...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,DRMT,Transit Intercity and Rail Capital Program,"November 19, 2021","March 3, 2022",3,2 Years,500000000,Public agencies and joint power agencies,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,DRMT,5307 Urbanized Area Formula Grants,Missing,Missing,Missing,1 Year,5375259282,MPOs and Transit Agencies,Formula,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,73,Federal Highway Administration (DOT),"Grants for Planning, Feasibility Analysis, and...",Missing,Missing,Missing,Period of Availability: 4 year,20000000,"State, Metropolitan Planning Organization (w/ ...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,74,Federal Highway Administration (DOT),Strategic Innovation for Revenue Collection\r\...,Missing,Missing,Missing,Available until expended\r,15000000,"States, Metropolitan Planning Organizations, a...","Grants, Cooperative Agreements, other contracts\r",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,75,Federal Transit Administration (DOT),Bus Exportable Power Systems,"April 12, 2022","June 13, 2022",2,Missing,1000000,"Providers of public transportation, including ...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,76,Federal Transit Administration (DOT),Enhancing Mobility Innovation,"November 12, 2021","January 11, 2022",1,Missing,2000000,"Providers of public transportation, including ...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df = df.rename(columns={'grant_type__formula,_competitive,_both,_discretionary_n_a_':'distribution_type',
                       'current__in_this_fy__or_expired':'current'})

In [58]:
df.sample()

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,unnamed:_27,unnamed:_28,unnamed:_29,unnamed:_30,unnamed:_31,unnamed:_32,unnamed:_33,unnamed:_34,unnamed:_35,unnamed:_36
49,50,Federal Highway Administration (DOT),Highway Research & Development Program,TBD,Missing,Missing,Available until expended,310000000,Varies,Grant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 37 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   grant_#                                             76 non-null     int64  
 1   division                                            76 non-null     object 
 2   grant_program                                       76 non-null     object 
 3   notice_of_funding                                   76 non-null     object 
 4   fy_due_date                                         76 non-null     object 
 5   months_to_apply__c_d_                               76 non-null     object 
 6   fy_cycle                                            76 non-null     object 
 7   funding_amount_available                            76 non-null     int64  
 8   agency_eligibility                                  76 non-null     object 
 9   g

In [60]:
df.federal_or_state_funded.value_counts()

Federal    59
State      12
Both        4
Missing     1
Name: federal_or_state_funded, dtype: int64

In [61]:
(
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)

,division,federal_or_state_funded,avg_funds
0,Clean CA,State,2.96e+08
1,DLA,State,1.27e+07
2,DRMT,State,9.16e+07
3,SB1,State,4.78e+08
4,SB1/DLA,State,5.79e+08


In [62]:
chart = (
    alt.Chart(
        (
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [63]:
chart = (
    alt.Chart(
       (
    df
    >>filter(_.federal_or_state_funded=='Federal')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [64]:
df.sample(10)

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,unnamed:_27,unnamed:_28,unnamed:_29,unnamed:_30,unnamed:_31,unnamed:_32,unnamed:_33,unnamed:_34,unnamed:_35,unnamed:_36
37,38,Federal Highway Administration (DOT),Ferry Programs: Electric or Low Emitting Ferry...,Missing,Missing,Missing,1 Year,50000000,"Unknown - only says ""Will be identified in a N...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,77,Federal Transit Administration (DOT),Public Transportation on Indian Reservations P...,"February 16, 2022","May 25, 2022",3,1 Year,87500000,Federally recognized American Indian or Alaska...,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,16,SB1/DLA,Active Transportation Program,"March 16, 2022","June 15, 2022",2,1 Year,651000000,"Local, Regional, or State Agencies, Caltrans, ...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,DRMT,5307 Urbanized Area Formula Grants,Missing,Missing,Missing,1 Year,5375259282,MPOs and Transit Agencies,Formula,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,DRMT,State of Good Repair,"July 1, 2022","September 1, 2021",Missing,1 Year,105000000,Transit operators,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,17,SB1,State Highway Operation and Protection Program,Missing,"January 31, 2022",Missing,2 Years,4325000000,"RTPA, Counties",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,DRMT,Rural Transit & Intercity Bus 5311 & 5311f (a...,"March 1, 2022","April 29, 2022",1,2 Years,781000000,"Public transit providers, state and local gove...",Formula,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,15,SB1,Local Partnership Program,"August 17, 2022","November 28, 2022",3,3 Years,200000000,"City, County,",Both,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,SB1,Trade Corridor Enhancement Program,"August 17, 2022","November 18, 2022",3,2 Years,800000000,"Caltrans, Regional Agencies, Counties, Cities,...",Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,24,DLA,Local Highway Safety Improvement Program (HSIP),"May 9, 2022","September 12, 2022",4,2 Years (sometimes 1),227600000,"City, a County, or a Tribal Government",First Come First Serve Basis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df>>group_by(_.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,division,avg_funds
0,Caltrans - Unknown Division,2.30e+09
1,Clean CA,2.96e+08
2,DLA,1.14e+08
3,DLA,0.00e+00
4,DRMT,1.30e+09
5,Federal Highway Administration (DOT),4.22e+08
6,Federal Railroad Administration (DOT),1.00e+09
7,Federal Transit Administration (DOT),5.25e+08
8,Office of the Secretary (DOT),2.07e+09
9,SB1,8.56e+08


In [66]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [67]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [68]:
df>>group_by(_.grant_type, _.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,grant_type,division,avg_funds
0,Both,DRMT,6.04e+08
1,Both,SB1,1.17e+08
2,Competitive,Caltrans - Unknown Division,2.30e+09
3,Competitive,DLA,1.35e+08
4,Competitive,DRMT,2.50e+09
5,Competitive,Federal Highway Administration (DOT),4.63e+08
6,Competitive,Federal Railroad Administration (DOT),1.00e+09
7,Competitive,Federal Transit Administration (DOT),4.48e+08
8,Competitive,Office of the Secretary (DOT),2.44e+09
9,Competitive,SB1,1.72e+09


In [69]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)

In [70]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type (Nation-wide)")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)